In [32]:
# 2번 셀: 기본 import + PaddleOCR 초기화

import cv2
import numpy as np
import pyperclip
import mss

from paddleocr import PaddleOCR

# 전역 OCR 엔진 (한 번만 생성)
ocr = PaddleOCR(
    use_angle_cls=False,   # 코드 스샷이면 보통 회전 X
    lang="en",             # 코드면 en으로 충분
)

print("PaddleOCR 초기화 완료")



C:\Users\snk74\AppData\Local\Temp\ipykernel_34788\3317341365.py:11: DeprecationWarning: The parameter `use_angle_cls` has been deprecated and will be removed in the future. Please use `use_textline_orientation` instead.
  ocr = PaddleOCR(
Creating model: ('PP-LCNet_x1_0_doc_ori', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\snk74\.paddlex\official_models\PP-LCNet_x1_0_doc_ori`.
Creating model: ('UVDoc', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\snk74\.paddlex\official_models\UVDoc`.
Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\snk74\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('en_PP-OCRv5_mobile_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users

PaddleOCR 초기화 완료


In [33]:
# 3번 셀: 전체 화면 캡처 함수 + 마우스 ROI 선택

def capture_full_screen_bgr():
    """
    mss로 전체 화면을 캡처하여 BGR(OpenCV) 이미지로 반환합니다.
    """
    with mss.mss() as sct:
        monitor = sct.monitors[1]  # 첫 번째 모니터 전체
        sct_img = sct.grab(monitor)
        img = np.array(sct_img)  # BGRA
        img_bgr = cv2.cvtColor(img, cv2.COLOR_BGRA2BGR)
    return img_bgr

def select_roi(img_bgr, win_name="Crop"):
    """
    OpenCV 창에서 마우스로 ROI를 선택하고 잘라낸 이미지를 반환합니다.
    """
    clone = img_bgr.copy()
    cv2.namedWindow(win_name, cv2.WINDOW_NORMAL)
    cv2.imshow(win_name, clone)
    r = cv2.selectROI(win_name, clone, showCrosshair=True, fromCenter=False)
    cv2.destroyWindow(win_name)

    x, y, w, h = r
    if w == 0 or h == 0:
        return None  # 사용자가 취소한 경우
    crop = img_bgr[y:y+h, x:x+w]
    return crop

print("캡처 & ROI 선택 함수 준비 완료")


캡처 & ROI 선택 함수 준비 완료


In [34]:
import numpy as np

def ocr_image_with_paddle(img_bgr):
    """
    OpenCV BGR 이미지를 받아:
    1) 전처리 후
    2) PaddleOCR 3.x predict로 인식하고
    3) 줄 + 들여쓰기를 어느 정도 반영한 코드 문자열로 반환.
    """
    # 1) 전처리
    img_proc = preprocess_for_paddleocr(img_bgr)

    # 2) OCR 실행
    result = ocr.predict(
        input=img_proc,
        use_doc_orientation_classify=False,
        use_doc_unwarping=False,
        use_textline_orientation=False,
    )

    if not result or len(result) == 0:
        return ""

    r = result[0]

    # OCRResult는 dict처럼 동작하니 .get 사용
    data = r if hasattr(r, "get") else dict(r)

    texts   = data.get("rec_texts", [])
    boxes   = data.get("rec_boxes", None)
    scores  = data.get("rec_scores", [])
    # 길이 맞춰주기 (혹시 모를 어긋남 방지)
    if not scores or len(scores) != len(texts):
        scores = [1.0] * len(texts)

    # 3) 신뢰도 낮은 항목은 버리기 (예: score < 0.5)
    score_thresh = 0.5
    items = []
    if boxes is None or len(boxes) != len(texts):
        # 박스 정보가 없으면, 일단 줄 정보 없이 텍스트만 반환
        filtered = [t for t, s in zip(texts, scores)
                    if t and t.strip() and s >= score_thresh]
        return "\n".join(filtered)

    for text, box, score in zip(texts, boxes, scores):
        if not text or not text.strip():
            continue
        if score < score_thresh:
            continue

        x_min, y_min, x_max, y_max = box
        items.append((float(x_min), float(y_min), text))

    if not items:
        return ""

    # 4) y → x 순서로 정렬
    items.sort(key=lambda x: (x[1], x[0]))

    # 5) y좌표 기준으로 줄 그룹핑 + 들여쓰기 복원
    lines = []
    current_line = []
    current_y = None
    y_threshold = 12     # 줄 간 간격 기준 (픽셀)
    space_px    = 12     # 공백 1개에 해당하는 픽셀 수 (조절 가능)

    for x_min, y_min, text in items:
        if current_y is None:
            current_y = y_min
            current_line = [(x_min, text)]
            continue

        if abs(y_min - current_y) <= y_threshold:
            current_line.append((x_min, text))
        else:
            # 이전 줄 마무리
            current_line.sort(key=lambda x: x[0])
            # 줄 내 최소 x 기준으로 상대적인 들여쓰기 계산
            line_min_x = current_line[0][0]
            parts = []
            for x, t in current_line:
                indent_level = int(max(x - line_min_x, 0) // space_px)
                parts.append(" " * indent_level + t)
            lines.append(" ".join(parts))

            # 새 줄 시작
            current_y = y_min
            current_line = [(x_min, text)]

    # 마지막 줄 처리
    if current_line:
        current_line.sort(key=lambda x: x[0])
        line_min_x = current_line[0][0]
        parts = []
        for x, t in current_line:
            indent_level = int(max(x - line_min_x, 0) // space_px)
            parts.append(" " * indent_level + t)
        lines.append(" ".join(parts))

    return "\n".join(lines)

print("업데이트된 ocr_image_with_paddle 준비 완료")





업데이트된 ocr_image_with_paddle 준비 완료


In [35]:
# 전처리 함수: 코드 스크린샷에 맞춘 기본 세팅


def preprocess_for_paddleocr(img_bgr):
    """
    코드 스크린샷용 기본 전처리:
    - 그레이스케일
    - 해상도 작을 경우 2배 확대
    - OTSU 이진화로 대비 강화
    - 다시 BGR로 변환 (PaddleOCR는 3채널 이미지 선호)
    """
    # 1) 그레이스케일
    gray = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2GRAY)

    h, w = gray.shape[:2]

    # 2) 너무 작으면 2배 확대
    scale = 1
    if min(h, w) < 600:
        scale = 2

    if scale != 1:
        gray = cv2.resize(
            gray,
            (w * scale, h * scale),
            interpolation=cv2.INTER_CUBIC,
        )

    # 3) OTSU 이진화로 대비 올리기
    _, th = cv2.threshold(
        gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU
    )

    # 4) 다시 BGR로 변환
    proc_bgr = cv2.cvtColor(th, cv2.COLOR_GRAY2BGR)
    return proc_bgr

print("preprocess_for_paddleocr 준비 완료")


preprocess_for_paddleocr 준비 완료


In [36]:
# 5번 셀: 단계별 디버그가 들어간 전체 파이프라인

print("=== 1) 전체 화면 캡처 시작 ===")
full_img = capture_full_screen_bgr()

print("full_img 타입:", type(full_img))
if isinstance(full_img, np.ndarray):
    print("full_img shape:", full_img.shape)
else:
    print("full_img가 numpy 배열이 아닙니다!")

# full_img가 제대로 안 들어왔으면 여기서 중단
if not isinstance(full_img, np.ndarray):
    raise ValueError("full_img가 유효한 이미지가 아닙니다 (numpy 배열 아님).")

if full_img.size == 0:
    raise ValueError("full_img가 비어 있습니다 (size==0). 캡처 단계에서 문제가 있습니다.")

print("\n=== 2) ROI 선택 시작 ===")
crop_img = select_roi(full_img, win_name="Select Code Region")

if crop_img is None:
    print("ROI가 선택되지 않았습니다. (w=0 또는 h=0) 다시 시도해 주세요.")
else:
    print("crop_img 타입:", type(crop_img))
    if isinstance(crop_img, np.ndarray):
        print("crop_img shape:", crop_img.shape, "size:", crop_img.size)
    else:
        print("crop_img가 numpy 배열이 아닙니다!")

    if not isinstance(crop_img, np.ndarray):
        raise ValueError("crop_img가 유효한 이미지가 아닙니다 (numpy 배열 아님).")

    if crop_img.size == 0:
        raise ValueError("crop_img가 비어 있습니다 (size==0). ROI 선택에서 문제가 있습니다.")

    print("\n=== 3) PaddleOCR로 인식 시도 ===")
    try:
        text = ocr_image_with_paddle(crop_img)
    except Exception as e:
        print(">>> OCR 단계에서 예외 발생!")
        print("예외 타입:", type(e))
        print("예외 내용:", e)
        # 여기서 스택트레이스까지 보고 싶으면:
        import traceback
        traceback.print_exc()
        raise  # 다시 던져서 Jupyter가 ValueError 위치를 보여주도록

    print("\n=== 4) OCR 결과 출력 ===")
    print("──────── OCR RESULT ────────")
    print(text)
    print("──────── END ───────────────")

    print("\n=== 5) 클립보드 복사 ===")
    try:
        pyperclip.copy(text)
        print("텍스트를 클립보드에 복사했습니다.")
    except Exception as e:
        print("클립보드 복사 중 예외 발생:", e)


=== 1) 전체 화면 캡처 시작 ===


full_img 타입: <class 'numpy.ndarray'>
full_img shape: (1080, 1920, 3)

=== 2) ROI 선택 시작 ===
crop_img 타입: <class 'numpy.ndarray'>
crop_img shape: (91, 734, 3) size: 200382

=== 3) PaddleOCR로 인식 시도 ===

=== 4) OCR 결과 출력 ===
──────── OCR RESULT ────────
# full img ch  ot  oEt
if not isinstance(full img, np.ndarray):
raise ValueError("full_img  o|o|x| ofdt (numpy  o).")
──────── END ───────────────

=== 5) 클립보드 복사 ===
텍스트를 클립보드에 복사했습니다.


In [37]:
# 디버그용: crop_img가 이미 위에서 만들어져 있다고 가정

img_for_ocr = crop_img  # 전처리 없이 그대로 사용

dbg_result = ocr.predict(
    input=img_for_ocr,
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False,
)

print("결과 개수:", len(dbg_result))
for i, r in enumerate(dbg_result):
    print(f"--- Result #{i} ---")
    # 1) 객체 타입
    print("type:", type(r))
    # 2) 내부 딕셔너리형 결과가 있으면 그걸 찍어보기
    if hasattr(r, "res"):
        print("r.res:", r.res)
    else:
        # fallback: 그냥 r 자체를 출력
        print("r:", r)




결과 개수: 1
--- Result #0 ---
type: <class 'paddlex.inference.pipelines.ocr.result.OCRResult'>
r: {'input_path': None, 'page_index': None, 'doc_preprocessor_res': {'output_img': array([[[24, ..., 24],
        ...,
        [24, ..., 24]],

       ...,

       [[24, ..., 24],
        ...,
        [24, ..., 24]]], shape=(91, 734, 3), dtype=uint8)}, 'dt_polys': [array([[ 2,  3],
       ...,
       [ 2, 25]], shape=(4, 2), dtype=int16), array([[ 1, 26],
       ...,
       [ 1, 48]], shape=(4, 2), dtype=int16), array([[41, 49],
       ...,
       [41, 72]], shape=(4, 2), dtype=int16)], 'model_settings': {'use_doc_preprocessor': False, 'use_textline_orientation': False}, 'text_det_params': {'limit_side_len': 64, 'limit_type': 'min', 'thresh': 0.3, 'max_side_limit': 4000, 'box_thresh': 0.6, 'unclip_ratio': 1.5}, 'text_type': 'general', 'text_rec_score_thresh': 0.0, 'return_word_box': False, 'rec_texts': ['# full_img7t Tllch otmolotEod7|a bEt', 'if not isinstance(full_img, np.ndarray):', 'raise Va